# nteract - open issue and open pr counts
Send a graphql query to GitHub
and work with files for reports.

Supports Python 3.6+

In [1]:
import csv
import json
import os
import pprint

import requests

In [2]:
# get api token and set authorization
api_token = os.environ['GITHUB_API_TOKEN']
headers = {'Authorization': f'token {api_token}'}

In [3]:
# set url to a graphql endpoint
url = 'https://api.github.com/graphql'

In [4]:
# add a json query
query = """
{
  organization(login: "nteract") {
    repositories(first: 80) {
      nodes {
        name
        url
        issues(states: OPEN) {
          totalCount
        }
        pullRequests(states: OPEN) {
          totalCount
        }
      }
    }
  }
}
"""

### Make request and create json and csv files

In [5]:
# submit the request
r = requests.post(url=url, json={'query': query}, headers=headers)

In [6]:
# create a json file from response
with open('data.json', 'w') as f:
    json.dump(r.json(), f)

In [7]:
# unpack the layers of json
nodes = r.json()['data']['organization']['repositories']['nodes']

unpacked = []
for node in nodes:
    unpacked.append(node)

In [8]:
headers = ['name', 'url', 'issues', 'prs']

rows = []
for obj in unpacked:
    new_dict = {'name':obj['name'], 'url':obj['url'], 'issues':obj['issues']['totalCount'], 'prs':obj['pullRequests']['totalCount']}
    rows.append(new_dict)

In [9]:
with open('mydata.csv', 'w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

Check file

In [10]:
#%%bash

#less mydata.csv

## Bring into pandas

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('mydata.csv')

In [13]:
df.columns

Index(['name', 'url', 'issues', 'prs'], dtype='object')

In [14]:
# df.head()

Generate basic report of total open issues

In [15]:
# df.dtypes

In [16]:
# df.index

In [17]:
# df.values

### Reports

In [18]:
# By repo name
df.sort_values(by=['name'])

,name,url,issues,prs
31,ansi-to-react,https://github.com/nteract/ansi-to-react,3,1
34,assets,https://github.com/nteract/assets,0,0
35,associator,https://github.com/nteract/associator,0,0
58,cabinet,https://github.com/nteract/cabinet,0,1
14,commutable,https://github.com/nteract/commutable,0,0
54,commutable-models,https://github.com/nteract/commutable-models,0,0
49,commutable-perf,https://github.com/nteract/commutable-perf,1,0
52,commuter,https://github.com/nteract/commuter,28,1
23,design,https://github.com/nteract/design,3,0
56,desktop-manual,https://github.com/nteract/desktop-manual,0,0


In [19]:
# By open issue count
df.sort_values(by=['issues'], ascending=False).head(10)


,name,url,issues,prs
3,nteract,https://github.com/nteract/nteract,248,1
1,hydrogen,https://github.com/nteract/hydrogen,68,3
52,commuter,https://github.com/nteract/commuter,28,1
57,papermill,https://github.com/nteract/papermill,11,1
43,nteract.io,https://github.com/nteract/nteract.io,9,1
53,snakestagram,https://github.com/nteract/snakestagram,8,0
47,rx-jupyter,https://github.com/nteract/rx-jupyter,8,0
60,vdom,https://github.com/nteract/vdom,7,2
0,sidecar,https://github.com/nteract/sidecar,6,1
4,nteract.github.io,https://github.com/nteract/nteract.github.io,5,0


In [20]:
# by open pr count
df.sort_values(by=['prs'], ascending=False)

,name,url,issues,prs
1,hydrogen,https://github.com/nteract/hydrogen,68,3
60,vdom,https://github.com/nteract/vdom,7,2
19,ick,https://github.com/nteract/ick,3,1
58,cabinet,https://github.com/nteract/cabinet,0,1
57,papermill,https://github.com/nteract/papermill,11,1
52,commuter,https://github.com/nteract/commuter,28,1
45,inodejs,https://github.com/nteract/inodejs,0,1
43,nteract.io,https://github.com/nteract/nteract.io,9,1
36,notebook-preview,https://github.com/nteract/notebook-preview,3,1
33,mathjax-electron,https://github.com/nteract/mathjax-electron,1,1


In [21]:
# output data to a csv
# df.to_csv('issue_report.csv')